In [286]:
import wrangle as w 
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from matplotlib.ticker import ScalarFormatter
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

First I need to modify the dataframe and make a separate version of it for modeling; split, scale, fit_transform, make dataframes have the same columns (actor dummies may complicate this)

In [287]:
df = w.wrangle_df()

Using cached CSV


I originally wanted to one-hot encode for every instance of cast_actor_1, 2, and 3, but the resultant shape is so large as to become unworkable (at least in a reasonable time span.

In [288]:
df.production_company.value_counts().head(10)

UniversalPictures                     398
ParamountPictures                     386
TwentiethCenturyFoxFilmCorporation    291
ColumbiaPictures                      232
NewLineCinema                         184
WarnerBros.                           162
WaltDisneyPictures                    145
Metro-Goldwyn-Mayer(MGM)              107
ColumbiaPicturesCorporation           104
UnitedArtists                         101
Name: production_company, dtype: int64

In [289]:
threshold = 101
df.loc[df['production_company'].value_counts()\
       [df['production_company']].values < threshold, 'production_company'] = "other_company"

In [290]:
df.production_company.value_counts()

other_company                         4783
UniversalPictures                      398
ParamountPictures                      386
TwentiethCenturyFoxFilmCorporation     291
ColumbiaPictures                       232
NewLineCinema                          184
WarnerBros.                            162
WaltDisneyPictures                     145
Metro-Goldwyn-Mayer(MGM)               107
ColumbiaPicturesCorporation            104
UnitedArtists                          101
Name: production_company, dtype: int64

In [291]:
df.cast_actor_1.value_counts().head(10)

Nicolas Cage          41
Robert De Niro        37
Bruce Willis          36
Sylvester Stallone    35
Johnny Depp           34
Denzel Washington     33
Tom Hanks             33
Clint Eastwood        32
Tom Cruise            30
Robin Williams        26
Name: cast_actor_1, dtype: int64

In [292]:
threshold = 26
df.loc[df['cast_actor_1'].value_counts()\
       [df['cast_actor_1']].values < threshold, 'cast_actor_1'] = "other_actor"

In [293]:
df.cast_actor_1.value_counts()

other_actor              6452
Nicolas Cage               41
Robert De Niro             37
Bruce Willis               36
Sylvester Stallone         35
Johnny Depp                34
Tom Hanks                  33
Denzel Washington          33
Clint Eastwood             32
Tom Cruise                 30
Arnold Schwarzenegger      26
Kevin Costner              26
Robin Williams             26
John Travolta              26
Adam Sandler               26
Name: cast_actor_1, dtype: int64

In [294]:
dummy_group = ['cast_actor_1', 'production_company']
dummy_df = pd.get_dummies(df.loc[:,dummy_group])

In [295]:
keep =  ['budget','runtime', 'vote_average','vote_count', 'success', 
         'release_year', 'is_genre_adventure', 'is_genre_horror', 
         'is_genre_drama', 'is_genre_scifi', 'is_genre_romance',
         'is_genre_thriller', 'is_genre_crime', 'is_genre_comedy',
         'is_genre_animation', 'is_genre_action', 'is_genre_mystery',
         'is_genre_fantasy', 'is_genre_documentary', 'total_n_cast']
# success will be the only one of these that is not used as a predictor, given that it is our target
# I'm not sure if revenue would be appropriate to include. 

In [296]:
modeling_df = df.loc[:,keep]

In [297]:
# Concatenate dummy dataframe to original 
modeling_df = pd.concat([modeling_df, dummy_df], axis=1)

In [298]:
modeling_df.shape

(6893, 46)

In [299]:
modeling_df.columns

Index(['budget', 'runtime', 'vote_average', 'vote_count', 'success',
       'release_year', 'is_genre_adventure', 'is_genre_horror',
       'is_genre_drama', 'is_genre_scifi', 'is_genre_romance',
       'is_genre_thriller', 'is_genre_crime', 'is_genre_comedy',
       'is_genre_animation', 'is_genre_action', 'is_genre_mystery',
       'is_genre_fantasy', 'is_genre_documentary', 'total_n_cast',
       'cast_actor_1_Adam Sandler', 'cast_actor_1_Arnold Schwarzenegger',
       'cast_actor_1_Bruce Willis', 'cast_actor_1_Clint Eastwood',
       'cast_actor_1_Denzel Washington', 'cast_actor_1_John Travolta',
       'cast_actor_1_Johnny Depp', 'cast_actor_1_Kevin Costner',
       'cast_actor_1_Nicolas Cage', 'cast_actor_1_Robert De Niro',
       'cast_actor_1_Robin Williams', 'cast_actor_1_Sylvester Stallone',
       'cast_actor_1_Tom Cruise', 'cast_actor_1_Tom Hanks',
       'cast_actor_1_other_actor', 'production_company_ColumbiaPictures',
       'production_company_ColumbiaPicturesCorporatio

The data types are as they should be, as far as I know. 

In [300]:
train, validate, test = w.train_validate_test_split(modeling_df)

In [301]:
print('train:', train.shape)
print('validate:', validate.shape)
print('test:', test.shape)

train: (3859, 46)
validate: (1655, 46)
test: (1379, 46)


***
Time to scale

In [302]:
X_train = train.drop(columns=['success'])
y_train = train['success']

X_validate = validate.drop(columns=['success'])
y_validate = validate['success']

X_test = test.drop(columns=['success'])
y_test = test['success']

In [303]:
# Now I need to scale. 
needs_scaling = ['budget','runtime', 'vote_average',
                 'vote_count','release_year', 'total_n_cast']

In [304]:
#scaler = MinMaxScaler()
## Note that we only call .fit with the training data,
## but we use .transform to apply the scaling to all the data splits.
#scaler.fit(X_train[needs_scaling])
#
#X_train_scaled = pd.DataFrame(scaler.transform(X_train[needs_scaling]))
#X_validate_scaled = pd.DataFrame(scaler.transform(X_validate[needs_scaling]))
#X_test_scaled = pd.DataFrame(scaler.transform(X_test[needs_scaling]))

scaler = MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns = X_train.columns)
X_validate = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns = X_validate.columns)
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns = X_test.columns)

In [305]:
y_train.value_counts()

0    2358
1    1501
Name: success, dtype: int64

In [306]:
X_train['baseline_prediction'] = 0
X_validate['baseline_prediction'] = 0
X_test['baseline_prediction'] = 0

In [307]:
X_train.shape

(3859, 46)

In [308]:
y_train.shape

(3859,)

In [311]:
# in case I want it for later
#train_scaled = X_train.merge(ytdf, how='left', on=X_train.index)
#validate_scaled = X_validate.merge(yvdf, how='left', left_index = True, right_index = True)
#train_scaled = X_train.merge(ytedf, how='left', left_index = True, right_index = True)
train_scaled = X_train.copy()
train_scaled['success'] = y_train

validate_scaled = X_validate.copy()
validate_scaled['success'] = y_validate

test_scaled = X_test.copy()
test_scaled['success'] = y_test

In [312]:
print('train:', train_scaled.shape)
print('validate:', validate_scaled.shape)
print('test:', test_scaled.shape)

train: (3859, 47)
validate: (1655, 47)
test: (1379, 47)


In [43]:
# print grid search results
def print_cv_results(gs, title):

    print(title)

    print(f'Best Score = {gs.best_score_:.4f}')
    print(f'Best Hyper-parameters = {gs.best_params_}')
    print()

    print('Test Scores:')
    test_means = gs.cv_results_['mean_test_score']
    test_stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(test_means, test_stds, gs.cv_results_['params']):
        print(f'{mean:.4f} (+/-{std:.3f}) for {params}')
    print()

    print('Training Scores:')
    train_means = gs.cv_results_['mean_train_score']
    train_stds = gs.cv_results_['std_train_score']
    for mean, std, params in zip(train_means, train_stds, gs.cv_results_['params']):
        print(f'{mean:.4f} (+/-{std:.3f}) for {params}')
        
        # save grid search results to file
def save_cv_results(gs, title, fileName):
    with open(fileName, 'a') as f:

        print(title, file=f)

        print(f'Best Score = {gs.best_score_:.4f}', file=f)
        print(f'Best Hyper-parameters = {gs.best_params_}', file=f)
        print('', file=f)

        print('Test Scores:', file=f)
        test_means = gs.cv_results_['mean_test_score']
        test_stds = gs.cv_results_['std_test_score']
        for mean, std, params in zip(test_means, test_stds, gs.cv_results_['params']):
            print(f'{mean:.4f} (+/-{std:.3f}) for {params}', file=f)
        print('', file=f)

        print('Training Scores:', file=f)
        train_means = gs.cv_results_['mean_train_score']
        train_stds = gs.cv_results_['std_train_score']
        for mean, std, params in zip(train_means, train_stds, gs.cv_results_['params']):
            print(f'{mean:.4f} (+/-{std:.3f}) for {params}', file=f)

In [45]:
# Logistic Regression
logReg = LogisticRegression()
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = {'C': c_list,
              'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}

gs = GridSearchCV(estimator=logReg,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  return_train_score=True)
#gs = gs.fit(X_dummy_train, y_train)
gs = gs.fit(X_train, y_train)
print_cv_results(gs, 'Logistic Regression Accuracy')

KeyboardInterrupt: 